In [3]:
import os
from langchain_huggingface import HuggingFaceEndpoint
import pandas as pd
import numpy as np

from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

import requests
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from huggingface_hub import InferenceClient
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import logging
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
# Define the model parameters
model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 500,
    "repetition_penalty": 1.01, #1.03
    "return_full_text": False,
    "seed": 42,
    # "stop": ["Human"],
    "temperature": 0.1,
    "top_k": 25,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True   #True
}


# Default prompt
default_prompt = """ YOU ARE A SKILLED SUMMARIZER, ABLE TO EXTRACT THE MOST IMPORTANT POINTS FROM A THAI TEXT AND PRESENT THEM IN A CLEAR AND CONCISE MANNER. YOU HAVE BEEN TRAINED ON A VARIETY OF TEXTS AND CAN IDENTIFY THE KEY ELEMENTS THAT SUMMARIZE THE MAIN IDEA.

###INSTRUCTIONS###

- READ THE THAI TEXT CAREFULLY AND IDENTIFY THE MAIN TOPIC OR IDEA.
- EXTRACT THE MOST IMPORTANT POINTS THAT SUMMARIZE THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
- PRESENT THE SUMMARY IN A CLEAR AND CONCISE MANNER, USING PROPER THAI LANGUAGE AND GRAMMAR.

###Chain of Thoughts###

1. **Comprehension:**
   1.1. READ THE THAI TEXT TO UNDERSTAND THE MAIN TOPIC OR IDEA.
   1.2. IDENTIFY THE KEY ELEMENTS THAT SUPPORT THE MAIN IDEA.

2. **Summary:**
   2.1. EXTRACT THE MOST IMPORTANT POINTS FROM THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
   2.2. ORGANIZE THE POINTS IN A LOGICAL AND COHERENT MANNER.

3. **Review:**
   3.1. CHECK THE SUMMARY FOR ACCURACY AND COMPLETENESS.
   3.2. ENSURE THE SUMMARY IS CLEAR, CONCISE, AND FREE OF ERRORS.

###What Not To Do###

OBEY and never do:
- **NEVER** INCLUDE UNNECESSARY DETAILS OR EXAMPLES IN THE SUMMARY.
- **DO NOT** OMIT KEY POINTS THAT ARE ESSENTIAL TO UNDERSTANDING THE MAIN IDEA.
- **NEVER** PROVIDE A SUMMARY THAT IS LONGER THAN NECESSARY OR CONTAINS ERRORS.
- **DO NOT** FAIL TO REVIEW THE SUMMARY FOR ACCURACY AND COMPLETENESS.

###Few-Shot Example###

**Input:** [Example Thai text]
**Output:** 
-[Main point 1]
-[Main point 2]
-[Main point 3]

**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**
**RESPONSE ONLY **MAIN POINT**

###THIS IS A MANDATORY DIRECTIVE###
### FORMAT ANSWER ###
-[Main point 1]
-[Main point 2]
-[Main point 3]

###RESPONSE IN **THAI LANGUAGE**###

{}


"""

client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')



import logging
import time

# Function to split text into chunks based on token limits
def split_text_to_fit_token_limit(text, max_tokens=18000):
    """Splits the text into smaller chunks that fit within the token limit."""
    words = text.split()  
    approx_tokens_per_word = 1.5 
    max_words = int(max_tokens / approx_tokens_per_word)
    
    # Split into chunks
    chunks = []
    current_chunk = []
    current_token_count = 0

    for word in words:
        word_token_count = len(word) * approx_tokens_per_word
        if current_token_count + word_token_count > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_token_count = word_token_count
        else:
            current_chunk.append(word)
            current_token_count += word_token_count

    # Add the last chunk if it contains any words
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\cpall_2565.txt'
text = read_text_file(file_path)
chunks = split_text_to_fit_token_limit(text)


# If you want to process chunks with a model or other logic, you can do that here
def generate_prompt(input_text, prompt=default_prompt, model_param=model_param, retries=3):
    chunks = split_text_to_fit_token_limit(input_text)
    results = []

    for chunk in chunks:
        for attempt in range(retries):
            try:
                formatted_prompt = prompt.format(chunk)
                output = client.text_generation(formatted_prompt, **model_param)
                results.append(output)
                break  # Break the retry loop if successful
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed: {e}")
                if "Model is overloaded" in str(e):
                    backoff_time = 2 ** attempt  
                    time.sleep(backoff_time)
                else:
                    raise  
        else:
            raise Exception("Max retries exceeded")

def generate_prompts(chunks, prompt_template, model_params):
    prompts = []
    for i, chunk in enumerate(chunks):
        prompt = prompt_template.format(chunk)
        prompts.append({
            "input": prompt,
            "params": model_params
        })
    return prompts

# Generate the prompts for each chunk of text
prompts = generate_prompts(chunks, default_prompt, model_param)

# Example usage: sending the first prompt to the client
# response = client.text_generation(prompts[0]['input'], **prompts[0]['params'])
# print(response)

responses = []
for prompt in prompts:
    response = client.text_generation(prompt['input'], **prompt['params'])
    responses.append(response)

# แสดงผลลัพธ์ทั้งหมด
for i, response in enumerate(responses):
    print(f"Response for chunk {i+1}:")
    print(response)
    print("\n-----------------------------------------------------------------------\n")

Response for chunk 1:
-[Main point 1]: การสื่อสาร ESG ของ CPL ในไตรมาสที่ 4 ของปี 2565 จะเน้นสื่อสารกับกลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปี และยังคงสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียสำคัญ
-[Main point 2]: CPR ได้รับการรับรองในฐานะองค์กรที่ยั่งยืนระดับโลกจากดัชนี DJSI World สำหรับกลุ่มอาหารและสินค้าอุปโภคบริโภคเป็นปีที่ 12 และยังได้รับการรับรอง THSI ต่อเนื่องเป็นปีที่ 5
-[Main point 3]: CPR ได้รับรางวัล 5 รางวัลจากเวที Asia Excellence Award ครั้งที่ 12 และยังได้รับรางวัล Read ESG 2022 และรางวัลองค์กรที่สนับสนุนคนพิการดีเด่น 6 ปีต่อเนื่อง
-[Main point 4]: การดำเนินการตามแนวปฏิบัติที่ยั่งยืนของ CPR ได้รับการรับรองจากเวทีต่างๆ และยังได้รับรางวัลสำหรับการสนับสนุนคนพิการ
-[Main point 5]: การสื่อสารผ่านช่องทางหลักและโซเชียลมีเดียเพื่อสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียสำคัญ
-[Main point 6]: การดำเนินการตามแนวปฏิบัติที่ยั่งยืนของ CPR ได้รับการรับรองจากเวทีต่างๆ และยังได้รับรางวัลสำหรับการสนับสนุนคนพิการ
-[Main point 7]: การสื่อสารผ่านช่องทาง

--------------------------------------------------

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging
import time

# กำหนดพารามิเตอร์ของโมเดล
model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 700,
    "repetition_penalty": 1.01,
    "return_full_text": False,
    "seed": 42,
    "temperature": 0.1,
    "top_k": 25,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True
}

# กำหนดโปรมต์เริ่มต้น
default_prompt = """ YOU ARE A SKILLED SUMMARIZER, ABLE TO EXTRACT THE MOST IMPORTANT POINTS FROM A THAI TEXT AND PRESENT THEM IN A CLEAR AND CONCISE MANNER. YOU HAVE BEEN TRAINED ON A VARIETY OF TEXTS AND CAN IDENTIFY THE KEY ELEMENTS THAT SUMMARIZE THE MAIN IDEA.

###INSTRUCTIONS###

- READ THE THAI TEXT CAREFULLY AND IDENTIFY THE MAIN TOPIC OR IDEA.
- EXTRACT THE MOST IMPORTANT POINTS THAT SUMMARIZE THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
- PRESENT THE SUMMARY IN A CLEAR AND CONCISE MANNER, USING PROPER THAI LANGUAGE AND GRAMMAR.

###Chain of Thoughts###

1. **Comprehension:**
   1.1. READ THE THAI TEXT TO UNDERSTAND THE MAIN TOPIC OR IDEA.
   1.2. IDENTIFY THE KEY ELEMENTS THAT SUPPORT THE MAIN IDEA.

2. **Summary:**
   2.1. EXTRACT THE MOST IMPORTANT POINTS FROM THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
   2.2. ORGANIZE THE POINTS IN A LOGICAL AND COHERENT MANNER.

3. **Review:**
   3.1. CHECK THE SUMMARY FOR ACCURACY AND COMPLETENESS.
   3.2. ENSURE THE SUMMARY IS CLEAR, CONCISE, AND FREE OF ERRORS.

###What Not To Do###

OBEY and never do:
- **NEVER** INCLUDE UNNECESSARY DETAILS OR EXAMPLES IN THE SUMMARY.
- **DO NOT** OMIT KEY POINTS THAT ARE ESSENTIAL TO UNDERSTANDING THE MAIN IDEA.
- **NEVER** PROVIDE A SUMMARY THAT IS LONGER THAN NECESSARY OR CONTAINS ERRORS.
- **DO NOT** FAIL TO REVIEW THE SUMMARY FOR ACCURACY AND COMPLETENESS.

###Few-Shot Example###

**Input:** [Example Thai text]
**Output:** 
-[MAIN POINT 1]
-[MAIN POINT 2]
-[MAIN POINT 3]

{}
**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**
**RESPONSE ONLY **MAIN POINT**

###THIS IS A MANDATORY DIRECTIVE###
### FORMAT ANSWER ###
-[MAIN POINT 1]

###RESPONSE IN **THAI LANGUAGE**###
"""


def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_text_using_recursive(text, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)  

def generate_prompts(chunks, prompt_template, model_params):
    prompts = []
    for chunk in chunks:
        prompt = prompt_template.format(chunk)
        prompts.append({
            "input": prompt,
            "params": model_params
        })
    return prompts

def generate_summary_responses(prompts, client, retries=3):
    responses = []
    for prompt in prompts:
        for attempt in range(retries):
            try:
                response = client.text_generation(prompt['input'], **prompt['params'])
                responses.append(response)
                break  # Break the retry loop if successful
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed: {e}")
                if "Model is overloaded" in str(e):
                    backoff_time = 2 ** attempt  
                    time.sleep(backoff_time)
                else:
                    raise  
        else:
            raise Exception("Max retries exceeded")
    return responses

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\ไฟล์ข้อความท่องเที่ยว.txt'
text = read_text_file(file_path)

chunks = split_text_using_recursive(text, chunk_size=10000, chunk_overlap=1000)
prompts = generate_prompts(chunks, default_prompt, model_param)

client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
responses = generate_summary_responses(prompts, client)

for i, response in enumerate(responses):
    print(f"Response for chunk {i+1}:")
    print(response)
    print("\n--------------------------------------------------\n")


Response for chunk 1:
-[ผู้ชายกำลังจะย้ายจากโตเกียวไปกรุงเทพฯ]
-[กำลังจะเดินทางโดยรถไฟและต้องซื้อข้าวกล่อง]
-[กำลังจะเดินทางไปต่างจังหวัดโดยรถไฟ]
-[กำลังจะซื้อของฝากและของกินบนรถไฟ]
-[กำลังจะเดินทางขึ้นภูเขาเพื่อไปที่พัก]
-[กำลังจะจบการถ่ายเทป]Human: 






































































































































































































































































































































































































































































































































































































--------------------------------------------------



In [25]:
# Define the model parameters
model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 500,
    "repetition_penalty": 1.01, #1.03
    "return_full_text": False,
    "seed": 42,
    # "stop": ["Human"],
    "temperature": 0.1,
    "top_k": 25,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True   #True
}


# Default prompt
default_prompt = """ YOU ARE A SKILLED SUMMARIZER, ABLE TO EXTRACT THE MOST IMPORTANT POINTS FROM A THAI TEXT AND PRESENT THEM IN A CLEAR AND CONCISE MANNER. YOU HAVE BEEN TRAINED ON A VARIETY OF TEXTS AND CAN IDENTIFY THE KEY ELEMENTS THAT SUMMARIZE THE MAIN IDEA.

###INSTRUCTIONS###

- READ THE THAI TEXT CAREFULLY AND IDENTIFY THE MAIN TOPIC OR IDEA.
- EXTRACT THE MOST IMPORTANT POINTS THAT SUMMARIZE THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
- PRESENT THE SUMMARY IN A CLEAR AND CONCISE MANNER, USING PROPER THAI LANGUAGE AND GRAMMAR.

###Chain of Thoughts###

1. **Comprehension:**
   1.1. READ THE THAI TEXT TO UNDERSTAND THE MAIN TOPIC OR IDEA.
   1.2. IDENTIFY THE KEY ELEMENTS THAT SUPPORT THE MAIN IDEA.

2. **Summary:**
   2.1. EXTRACT THE MOST IMPORTANT POINTS FROM THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
   2.2. ORGANIZE THE POINTS IN A LOGICAL AND COHERENT MANNER.

3. **Review:**
   3.1. CHECK THE SUMMARY FOR ACCURACY AND COMPLETENESS.
   3.2. ENSURE THE SUMMARY IS CLEAR, CONCISE, AND FREE OF ERRORS.

###What Not To Do###

OBEY and never do:
- **NEVER** INCLUDE UNNECESSARY DETAILS OR EXAMPLES IN THE SUMMARY.
- **DO NOT** OMIT KEY POINTS THAT ARE ESSENTIAL TO UNDERSTANDING THE MAIN IDEA.
- **NEVER** PROVIDE A SUMMARY THAT IS LONGER THAN NECESSARY OR CONTAINS ERRORS.
- **DO NOT** FAIL TO REVIEW THE SUMMARY FOR ACCURACY AND COMPLETENESS.

###Few-Shot Example###

**Input:** [Example Thai text]
**Output:** 
-[Main point 1]
-[Main point 2]
-[Main point 3]

**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**
**RESPONSE ONLY **MAIN POINT**

###THIS IS A MANDATORY DIRECTIVE###
### FORMAT ANSWER ###
-[Main point 1]
-[Main point 2]
-[Main point 3]

###RESPONSE IN **THAI LANGUAGE**###

{}
 

"""

client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')



import logging
import time

# Function to split text into chunks based on token limits
def split_text_to_fit_token_limit(text, max_tokens=18000):
    """Splits the text into smaller chunks that fit within the token limit."""
    words = text.split()  
    approx_tokens_per_word = 1.5 
    max_words = int(max_tokens / approx_tokens_per_word)
    
    # Split into chunks
    chunks = []
    current_chunk = []
    current_token_count = 0

    for word in words:
        word_token_count = len(word) * approx_tokens_per_word
        if current_token_count + word_token_count > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_token_count = word_token_count
        else:
            current_chunk.append(word)
            current_token_count += word_token_count

    # Add the last chunk if it contains any words
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\ไฟล์ข้อความ_2.txt'
text = read_text_file(file_path)
chunks = split_text_to_fit_token_limit(text)


# If you want to process chunks with a model or other logic, you can do that here
def generate_prompt(input_text, prompt=default_prompt, model_param=model_param, retries=3):
    chunks = split_text_to_fit_token_limit(input_text)
    results = []

    for chunk in chunks:
        for attempt in range(retries):
            try:
                formatted_prompt = prompt.format(chunk)
                output = client.text_generation(formatted_prompt, **model_param)
                results.append(output)
                break  # Break the retry loop if successful
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed: {e}")
                if "Model is overloaded" in str(e):
                    backoff_time = 2 ** attempt  
                    time.sleep(backoff_time)
                else:
                    raise  
        else:
            raise Exception("Max retries exceeded")

def generate_prompts(chunks, prompt_template, model_params):
    prompts = []
    for i, chunk in enumerate(chunks):
        prompt = prompt_template.format(chunk)
        prompts.append({
            "input": prompt,
            "params": model_params
        })
    return prompts

# Generate the prompts for each chunk of text
prompts = generate_prompts(chunks, default_prompt, model_param)

responses = []
for prompt in prompts:
    response = client.text_generation(prompt['input'], **prompt['params'])
    responses.append(response)

# แสดงผลลัพธ์ทั้งหมด
for i, response in enumerate(responses):
    print(f"Response for chunk {i+1}:")
    print(response)
    print("\n-----------------------------------------------------------------------\n")

Response for chunk 1:
-[รายการชวนเตี๋ยว]
-[ชวนลิ้มลองรสของก๋วยเตี๋ยวไทย]
-[จอร์จเป็นแขกในรายการ]
-[จอร์จไม่กินเครื่องใน]
-[จอร์จขอหมูล้วน]
-[จอร์จขอหมูล้วนสองที่]
-[จอร์จขอซุป]
-[จอร์จขอหมูกรอบ]
-[จอร์จขอไข่]
-[จอร์จขอก๋วยจั๊บ]
-[จอร์จขอเต้าหู้]
-[จอร์จขอราเม็ง]
-[จอร์จขอบะหมี่หมูแดง]
-[จอร์จขอราดหน้า]
-[จอร์จขอข้าวต้มแห้ง]
-[จอร์จขอก๋วยเตี๋ยวบูด]
-[จอร์จขอก๋วยจั๊บน้ำข้น]
-[จอร์จขอเต้าหู้ในก๋วยจั๊บ]
-[จอร์จขอราเม็งในก๋วยจั๊บ]
-[จอร์จขอบะหมี่หมูแดงในก๋วยจั๊บ]
-[จอร์จขอราดหน้าในก๋วยจั๊บ]
-[จอร์จขอข้าวต้มแห้งในก๋วยจั๊บ]
-[จอร์จขอก๋วยเตี๋ยวบูดในก๋วยจั๊บ]
-[จอร์จขอก๋วยจั๊บน้ำข้นในราเม็ง]
-[จอร์จขอเต้าหู้ในราเม็ง]
-[จอร์จขอราเม็งในราเม็ง]
-[จอร์จขอบะหมี่หมูแดงในราเม็ง]
-[จอร์จขอราดหน้าในราเม็ง]
-[จอร์จขอข้าวต้มแห้งในราเม็ง]
-[จอร์จขอก๋วยเตี๋ยวบูดในราเม็ง]
-[จอร์จขอก๋วยจั๊บน้ำข้นในบ

-----------------------------------------------------------------------



In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging
import time

# กำหนดพารามิเตอร์ของโมเดล
model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 800,
    "repetition_penalty": 1.01,
    "return_full_text": False,
    "seed": 42,
    "temperature": 0.1,
    "top_k": 25,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True
}

# กำหนดโปรมต์เริ่มต้น
default_prompt = """ YOU ARE A SKILLED SUMMARIZER, ABLE TO EXTRACT THE MOST IMPORTANT POINTS FROM A THAI TEXT AND PRESENT THEM IN A CLEAR AND CONCISE MANNER. YOU HAVE BEEN TRAINED ON A VARIETY OF TEXTS AND CAN IDENTIFY THE KEY ELEMENTS THAT SUMMARIZE THE MAIN IDEA.

###INSTRUCTIONS###

- READ THE THAI TEXT CAREFULLY AND IDENTIFY THE MAIN TOPIC OR IDEA.
- EXTRACT THE MOST IMPORTANT POINTS THAT SUMMARIZE THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
- PRESENT THE SUMMARY IN A CLEAR AND CONCISE MANNER, USING PROPER THAI LANGUAGE AND GRAMMAR.

###Chain of Thoughts###

1. **Comprehension:**
   1.1. READ THE THAI TEXT TO UNDERSTAND THE MAIN TOPIC OR IDEA.
   1.2. IDENTIFY THE KEY ELEMENTS THAT SUPPORT THE MAIN IDEA.

2. **Summary:**
   2.1. EXTRACT THE MOST IMPORTANT POINTS FROM THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
   2.2. ORGANIZE THE POINTS IN A LOGICAL AND COHERENT MANNER.

3. **Review:**
   3.1. CHECK THE SUMMARY FOR ACCURACY AND COMPLETENESS.
   3.2. ENSURE THE SUMMARY IS CLEAR, CONCISE, AND FREE OF ERRORS.

###What Not To Do###

OBEY and never do:
- **NEVER** INCLUDE UNNECESSARY DETAILS OR EXAMPLES IN THE SUMMARY.
- **DO NOT** OMIT KEY POINTS THAT ARE ESSENTIAL TO UNDERSTANDING THE MAIN IDEA.
- **NEVER** PROVIDE A SUMMARY THAT IS LONGER THAN NECESSARY OR CONTAINS ERRORS.
- **DO NOT** FAIL TO REVIEW THE SUMMARY FOR ACCURACY AND COMPLETENESS.

###Few-Shot Example###

**Input:** [Example Thai text]
**Output:** 
-[Main point 1]
-[Main point 2]
-[Main point 3]

**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**
**ANSWER ONLY **MAIN POINT**

{}

**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**
**ANSWER ONLY [MAIN POINT].**

### THIS IS A MANDATORY DIRECTIVE ###
### FORMAT ANSWER ###
- [MAIN POINT]

### RESPONSE IN **THAI LANGUAGE** ###
- **SUMMARIZE NO MORE THAN 5 POINTS.**

"""


def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_text_using_recursive(text, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)  

def generate_prompts(chunks, prompt_template, model_params):
    prompts = []
    for chunk in chunks:
        prompt = prompt_template.format(chunk)
        prompts.append({
            "input": prompt,
            "params": model_params
        })
    return prompts

def generate_summary_responses(prompts, client, retries=3):
    responses = []
    for prompt in prompts:
        for attempt in range(retries):
            try:
                response = client.text_generation(prompt['input'], **prompt['params'])
                responses.append(response)
                break  # Break the retry loop if successful
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed: {e}")
                if "Model is overloaded" in str(e):
                    backoff_time = 2 ** attempt  
                    time.sleep(backoff_time)
                else:
                    raise  
        else:
            raise Exception("Max retries exceeded")
    return responses

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\cpall_2565.txt'
text = read_text_file(file_path)

chunks = split_text_using_recursive(text, chunk_size=13185, chunk_overlap=1318)
prompts = generate_prompts(chunks, default_prompt, model_param)

client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
responses = generate_summary_responses(prompts, client)

for i, response in enumerate(responses):
    print(f"Response for chunk {i+1}:")
    print(response)
    print("\n--------------------------------------------------\n")

Response for chunk 1:
CPL ยังคงสื่อสาร ESG ผ่านกลยุทธ์สื่อสารใน Q4 ปี 2565 ให้ถึงกลุ่มคนรุ่นใหม่อายุน้อยกว่า 30 ปี และสร้างการรับรู้ในกลุ่มส่วนเกี่ยวข้องสำคัญผ่านช่องทางหลักและโซเชียลมีเดีย ได้รับรางวัลและเกียรติคุณต่างๆ ในปี 2565 อาทิ DJSI, THSI, และ Asia Excellence Awards. ยังมีกิจกรรมที่เน้นการพัฒนาความยั่งยืนและสนับสนุนคนพิการ ร่วมกับ 7-Eleven และองค์กรอื่นๆ ในเรื่องนวัตกรรมและเทคโนโลยี รวมถึงการสนับสนุนการศึกษาและเยาวชน. บริษัทยังมีการจัดประชุมผู้ถือหุ้นผ่านระบบออนไลน์ เพื่อป้องกันการแพร่ระบาดของโรค COVID-19 และมีการตรวจสอบคะแนนเสียงจากผู้สอบบัญชีภายนอก.

**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**

--------------------------------------------------

Response for chunk 2:
- [สรุปผลการดำเนินงานของบริษัทในปี 2565 ซึ่งมีรายได้รวมจากธุรกิจค้าปลีกเพิ่มขึ้น 45.1% และมีกำไรสุทธิ 13,272 ล้านบาท]
- [บริษัทมีการขยายสาขาร้านสะดวกซื้อ 7-11 704 สาขา]
- [บริษัทมีต้นทุนในการจัดจำหน่ายและค่าใช้จ่ายในการบริหารเพิ่มขึ้น 33.4% ซึ่งต่ำกว่าอัตราการเพิ่มขึ้นของรายได้]
- [บริษัทม

In [58]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging
import time

model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 500,
    "repetition_penalty": 1.01,
    "return_full_text": False,
    "seed": 42,
    "temperature": 0.1,
    "top_k": 25,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True
}

# กำหนดโปรมต์เริ่มต้น
default_prompt = """ YOU ARE A SKILLED SUMMARIZER, ABLE TO EXTRACT THE MOST IMPORTANT POINTS FROM A THAI TEXT AND PRESENT THEM IN A CLEAR AND CONCISE MANNER. YOU HAVE BEEN TRAINED ON A VARIETY OF TEXTS AND CAN IDENTIFY THE KEY ELEMENTS THAT SUMMARIZE THE MAIN IDEA.

###INSTRUCTIONS###

- READ THE THAI TEXT CAREFULLY AND IDENTIFY THE MAIN TOPIC OR IDEA.
- EXTRACT THE MOST IMPORTANT POINTS THAT SUMMARIZE THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
- PRESENT THE SUMMARY IN A CLEAR AND CONCISE MANNER, USING PROPER THAI LANGUAGE AND GRAMMAR.

###Chain of Thoughts###

1. **Comprehension:**
   1.1. READ THE THAI TEXT TO UNDERSTAND THE MAIN TOPIC OR IDEA.
   1.2. IDENTIFY THE KEY ELEMENTS THAT SUPPORT THE MAIN IDEA.

2. **Summary:**
   2.1. EXTRACT THE MOST IMPORTANT POINTS FROM THE TEXT, OMITTING ANY UNNECESSARY DETAILS.
   2.2. ORGANIZE THE POINTS IN A LOGICAL AND COHERENT MANNER.

3. **Review:**
   3.1. CHECK THE SUMMARY FOR ACCURACY AND COMPLETENESS.
   3.2. ENSURE THE SUMMARY IS CLEAR, CONCISE, AND FREE OF ERRORS.

###What Not To Do###

OBEY and never do:
- **NEVER** INCLUDE UNNECESSARY DETAILS OR EXAMPLES IN THE SUMMARY.
- **DO NOT** OMIT KEY POINTS THAT ARE ESSENTIAL TO UNDERSTANDING THE MAIN IDEA.
- **NEVER** PROVIDE A SUMMARY THAT IS LONGER THAN NECESSARY OR CONTAINS ERRORS.
- **DO NOT** FAIL TO REVIEW THE SUMMARY FOR ACCURACY AND COMPLETENESS.

###Few-Shot Example###

**Input:** [Example Thai text]
**Output:** 
-[Main point 1]
-[Main point 2]
-[Main point 3]

**WHEN SUMMARY POINT ALREADY, DO NOT REPLY ANYTHING ADDITIONALLY.**
**RESPONSE ONLY **MAIN POINT**

###THIS IS A MANDATORY DIRECTIVE###
### FORMAT ANSWER ###
-[MAIN POINT]

{}

###RESPONSE IN **THAI LANGUAGE**###
##SUMMARIZE MAIN IDEAA AND NO MORE THAN 5 MAIN POINTS.##


"""


def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def split_text_using_recursive(text, chunk_size, chunk_overlap):  
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)  

def generate_prompts(chunks, prompt_template, model_params):
    prompts = []
    for chunk in chunks:
        prompt = prompt_template.format(chunk)
        prompts.append({
            "input": prompt,
            "params": model_params
        })
    return prompts

def generate_summary_responses(prompts, client, retries=3):
    responses = []
    for prompt in prompts:
        for attempt in range(retries):
            try:
                response = client.text_generation(prompt['input'], **prompt['params'])
                responses.append(response)
                break  # Break the retry loop if successful
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed: {e}")
                if "Model is overloaded" in str(e):
                    backoff_time = 2 ** attempt  
                    time.sleep(backoff_time)
                else:
                    raise  
        else:
            raise Exception("Max retries exceeded")
    return responses

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\cpall_2565.txt'
text = read_text_file(file_path)

chunks = split_text_using_recursive(text, chunk_size=13185, chunk_overlap=1318)
prompts = generate_prompts(chunks, default_prompt, model_param)

client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
responses = generate_summary_responses(prompts, client)

for i, response in enumerate(responses):
    print(f"Response for chunk {i+1}:")
    print(response)
    print("\n--------------------------------------------------\n")

Response for chunk 1:
-[MAIN POINT]CPL ยังคงสื่อสาร ESG ผ่านกลยุทธ์สื่อสารในไตรมาสที่ 4 ปี 2565 ให้กลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปี และสร้างการรับรู้ในกลุ่มสื่อมวลชนสำคัญ
-[MAIN POINT]CPR ได้รับการรับรองความยั่งยืนระดับโลกจาก DJSI World สำหรับกลุ่มอาหารและของใช้ประจำวันเป็นปีที่ 12 และยังคงติดอันดับ THSI ปีที่ 5 ติดต่อกัน
-[MAIN POINT]CPR ได้รับรางวัล 5 รางวัลจากเวที Asia Excellence Award ครั้งที่ 12 และยังได้รับรางวัล Read ESG 2022 และรางวัลสนับสนุนคนพิการดีเด่น 6 ปีต่อเนื่อง
-[MAIN POINT]CPR และ 7-11 ร่วมมือในการจัดการขยะและสนับสนุนการพัฒนาชุมชน โดย 7-11 ได้ลดการใช้พลังงาน 54 ล้าน kWh หรือเท่ากับการปลูกต้นไม้ 5.4 แสนต้นต่อปี
-[MAIN POINT]CPR และ 7-11 ร่วมมือในการส่งเสริมการศึกษาและพัฒนายุวชน โดยการสนับสนุนการจัดการขยะและสนับสนุนการพัฒนาชุมชนผ่านโครงการต่างๆ และยังร่วมมือในการส่งเสริมการพัฒนายุวชนผ่านการสนับสนุนการศึกษาและโครงการพัฒนายุวชน. 

**[END OF RESPONSE]**



Assistant: -[MAIN POINT]CPL ยังคงสื่อสาร ESG ผ่านกลยุทธ์ส

--------------------------------------------------

Response

In [42]:
def calculate_chunk_parameters(text_length, desired_chunks, max_chunk_size, overlap_fraction=0.1):
    ideal_chunk_size = text_length // desired_chunks
    chunk_size = min(ideal_chunk_size, max_chunk_size)
    chunk_overlap = int(chunk_size * overlap_fraction)
    return chunk_size, chunk_overlap

text_length = len(text)
desired_chunks = 5  
max_chunk_size = 30000  # Define the maximum chunk size
overlap_fraction = 0.1  # 10% overlap between chunks

chunk_size, chunk_overlap = calculate_chunk_parameters(text_length, desired_chunks, max_chunk_size, overlap_fraction)

print(f"Calculated chunk size: {chunk_size}")
print(f"Calculated chunk overlap: {chunk_overlap}")

Calculated chunk size: 13185
Calculated chunk overlap: 1318


In [ ]:
help(client)